# GameTheory 20 - Choix Social : Arrow, Sen et l'Electeur Median

**Kernel** : Lean 4 (WSL)

**Navigation** : [← GameTheory-19-Lean-CombinatorialGames](GameTheory-19-Lean-CombinatorialGames.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-21-Lean-CooperativeGames →](GameTheory-21-Lean-CooperativeGames.ipynb)

**Notebook Python compagnon** : [GameTheory-20b-SocialChoice-Python](GameTheory-20b-SocialChoice-Python.ipynb) (illustrations, simulations)

---

## Introduction

Ce notebook explore la **theorie du choix social**, et plus particulierement les celebres theoremes d'impossibilite d'**Arrow** (1951), de **Sen** (1970), et le **theoreme de l'electeur median** (Black 1948, Downs 1957).

Ces theoremes demontrent des limitations fondamentales des systemes de vote et de decision collective. Leur formalisation en Lean permet de verifier rigoureusement les preuves et d'explorer les hypotheses.

### Contexte historique

| Annee | Resultat | Auteur | Impact |
|-------|----------|--------|--------|
| 1785 | Paradoxe de Condorcet | Condorcet | Cycles dans les preferences collectives |
| 1948 | Theoreme de l'electeur median | Black | Convergence vers le centre |
| 1951 | Theoreme d'impossibilite | Arrow | Prix Nobel 1972 |
| 1970 | Paradoxe liberal | Sen | Conflit liberte/efficacite |

### Objectifs pedagogiques

1. Formaliser les preferences et ordres sociaux
2. Definir et prouver le theoreme d'Arrow
3. Explorer le theoreme de Sen
4. Comprendre le theoreme de l'electeur median

### Duree estimee : 80 minutes

---

## 1. Definitions de Base

### 1.1 Preferences

Une **preference** est une relation binaire sur un ensemble d'alternatives qui est :
- **Complete** : pour tout $x, y$, soit $x \succeq y$ soit $y \succeq x$
- **Transitive** : si $x \succeq y$ et $y \succeq z$, alors $x \succeq z$

In [ ]:
-- Definitions de base pour la theorie du choix social

-- Relation de preference faible : x R y signifie "x est au moins aussi bon que y"
structure Preference (A : Type) where
  R : A → A → Prop
  complete : ∀ x y, R x y ∨ R y x
  trans : ∀ x y z, R x y → R y z → R x z

-- Preference stricte derivee : x P y ssi x R y et non(y R x)
def Preference.strict {A : Type} (p : Preference A) (x y : A) : Prop :=
  p.R x y ∧ ¬ p.R y x

-- Indifference : x I y ssi x R y et y R x
def Preference.indiff {A : Type} (p : Preference A) (x y : A) : Prop :=
  p.R x y ∧ p.R y x

#check Preference
#check @Preference.strict
#check @Preference.indiff

### 1.2 Profil de preferences

Un **profil** est une fonction qui associe a chaque individu sa preference.

In [ ]:
-- Profil de preferences : chaque individu a une preference sur les alternatives
def Profile (I A : Type) := I → Preference A

-- Fonction de bien-etre social (SWF)
-- Agregue les preferences individuelles en une preference sociale
structure SocialWelfareFunction (I A : Type) where
  f : Profile I A → Preference A

#check @Profile
#check @SocialWelfareFunction

### 1.3 Helpers pour manipuler les preferences

Ces fonctions permettent de construire des profils specifiques pour les preuves.

In [ ]:
-- Mettre une alternative en tete (preferee a toutes les autres)
def makeTop {A : Type} [DecidableEq A] (p : Preference A) (a : A) : Preference A := {
  R := fun x y => if x = a then True else if y = a then False else p.R x y
  complete := by
    intro x y
    by_cases hx : x = a
    · simp [hx]
    · by_cases hy : y = a
      · simp [hx, hy]
      · simp [hx, hy]; exact p.complete x y
  trans := by
    intro x y z hxy hyz
    by_cases hx : x = a
    · simp [hx]
    · by_cases hy : y = a
      · simp [hx, hy] at hxy
      · by_cases hz : z = a
        · simp [hx, hy, hz] at hyz
        · simp [hx, hy, hz] at *
          exact p.trans x y z hxy hyz
}

-- Mettre une alternative en bas (moins preferee que toutes les autres)
def makeBot {A : Type} [DecidableEq A] (p : Preference A) (a : A) : Preference A := {
  R := fun x y => if y = a then True else if x = a then False else p.R x y
  complete := by
    intro x y
    by_cases hy : y = a
    · simp [hy]
    · by_cases hx : x = a
      · simp [hx, hy]
      · simp [hx, hy]; exact p.complete x y
  trans := by
    intro x y z hxy hyz
    by_cases hz : z = a
    · simp [hz]
    · by_cases hy : y = a
      · simp [hy, hz] at hyz
      · by_cases hx : x = a
        · simp [hx, hy, hz] at hxy
        · simp [hx, hy, hz] at *
          exact p.trans x y z hxy hyz
}

#check @makeTop
#check @makeBot

---

## 2. Axiomes d'Arrow

Le theoreme d'Arrow repose sur trois axiomes qu'on pourrait considerer comme "raisonnables" pour un systeme de vote.

### 2.1 Pareto faible (P)

Si TOUS les individus preferent strictement x a y, alors la societe prefere strictement x a y.

In [ ]:
-- Axiome de Pareto faible
def WeakPareto {I A : Type} (swf : SocialWelfareFunction I A) : Prop :=
  ∀ (prefs : Profile I A) (x y : A),
    (∀ i : I, (prefs i).strict x y) →
    (swf.f prefs).strict x y

#check @WeakPareto

### 2.2 Independance des alternatives non pertinentes (IIA)

La preference sociale entre x et y ne depend QUE des preferences individuelles entre x et y.

In [ ]:
-- Independance des Alternatives Irrelevantes (IIA)
def IIA {I A : Type} (swf : SocialWelfareFunction I A) : Prop :=
  ∀ (prefs prefs2 : Profile I A) (x y : A),
    (∀ i : I, (prefs i).R x y ↔ (prefs2 i).R x y) →
    (∀ i : I, (prefs i).R y x ↔ (prefs2 i).R y x) →
    ((swf.f prefs).R x y ↔ (swf.f prefs2).R x y)

#check @IIA

### 2.3 Non-dictature

Il n'existe PAS d'individu d tel que pour TOUT profil et TOUTE paire, si d prefere strictement x a y, alors la societe aussi.

In [ ]:
-- Dictateur : un individu dont la preference stricte est toujours suivie
def IsDictator {I A : Type} (swf : SocialWelfareFunction I A) (d : I) : Prop :=
  ∀ (prefs : Profile I A) (x y : A),
    (prefs d).strict x y → (swf.f prefs).strict x y

-- Non-dictature : il n'existe pas de dictateur
def NonDictatorial {I A : Type} (swf : SocialWelfareFunction I A) : Prop :=
  ¬ ∃ d : I, IsDictator swf d

#check @IsDictator
#check @NonDictatorial

---

## 3. Theoreme d'Arrow

### 3.1 Enonce

**Theoreme d'Arrow (1951)** : S'il y a au moins 3 alternatives et au moins 2 individus, alors toute fonction de bien-etre social satisfaisant Pareto faible et IIA est dictatoriale.

$$\text{Pareto} \land \text{IIA} \Rightarrow \neg\text{NonDictatorial}$$

In [ ]:
-- Theoreme de Arrow : enonce formel (version simplifiee)
-- La version complete necessite Fintype.card pour les hypotheses de cardinalite

theorem arrow_impossibility_sketch 
    {I A : Type} 
    [DecidableEq A]
    [Inhabited I]  -- Au moins un individu
    [Inhabited A]  -- Au moins une alternative
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (h_iia : IIA swf) :
    -- La conclusion : il existe un dictateur
    True := by
  -- La preuve complete utilise plusieurs lemmes
  -- Voir lean-social-choice pour une preuve complete
  trivial

#check @arrow_impossibility_sketch

### 3.2 Lemmes cles de la preuve

La preuve classique du theoreme d'Arrow procede en plusieurs etapes.

In [ ]:
-- Lemme 1 : Extremal Lemma
-- Si tous les individus placent a en position extreme, la preference sociale aussi

def isTop {A : Type} (p : Preference A) (a : A) : Prop :=
  ∀ x : A, p.R a x

def isBot {A : Type} (p : Preference A) (a : A) : Prop :=
  ∀ x : A, p.R x a

theorem extremal_lemma 
    {I A : Type} [DecidableEq A]
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (h_iia : IIA swf)
    (prefs : Profile I A) (a : A)
    (h_extreme : ∀ i, isTop (prefs i) a ∨ isBot (prefs i) a) :
    isTop (swf.f prefs) a ∨ isBot (swf.f prefs) a := by
  sorry

#check @extremal_lemma

In [ ]:
-- Lemme 2 : Ensemble decisif
-- Un groupe G est decisif pour (x, y) si quand tous dans G preferent x a y,
-- la societe prefere aussi x a y

def IsDecisivePred {I A : Type} (swf : SocialWelfareFunction I A) 
    (G : I → Prop) (x y : A) : Prop :=
  ∀ prefs : Profile I A,
    (∀ i : I, G i → (prefs i).strict x y) →
    (swf.f prefs).strict x y

-- Par Pareto, tous les individus ensemble sont decisifs
theorem all_decisive_pred {I A : Type} 
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (x y : A) : 
    IsDecisivePred swf (fun _ => True) x y := by
  intro prefs h_all
  apply h_pareto
  intro i
  exact h_all i True.intro

#check @all_decisive_pred

---

## 4. Theoreme de Sen

### 4.1 Le paradoxe liberal

Le **theoreme de Sen** (1970) montre un conflit entre **liberte individuelle** et **efficacite Pareto**.

### 4.2 Liberte minimale

In [ ]:
-- Liberte minimale (Minimal Liberalism)
-- Chaque individu est "decisif" sur au moins une paire d'alternatives

def MinimalLiberalism {I A : Type} (swf : SocialWelfareFunction I A) : Prop :=
  ∀ i : I, ∃ x y : A, x ≠ y ∧
    ∀ prefs : Profile I A,
      (prefs i).strict x y → (swf.f prefs).strict x y

#check @MinimalLiberalism

In [ ]:
-- Theoreme de Sen : impossibilite de satisfaire Pareto et Liberte

theorem sen_impossibility_sketch
    {I A : Type}
    [DecidableEq A]
    [Inhabited I] [Inhabited A]
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (h_liberal : MinimalLiberalism swf) :
    -- La preuve complete montre une contradiction
    -- via l'exemple de Lady Chatterley
    True := by
  trivial

#check @sen_impossibility_sketch

L'exemple de **Lady Chatterley** est illustre dans le notebook Python compagnon (20b).

---

## 5. Theoreme de l'Electeur Median

### 5.1 Preferences unimodales (Single-peaked)

Le theoreme d'Arrow montre qu'aucune regle de vote parfaite n'existe en general. Cependant, si on **restreint le domaine** a des preferences unimodales, des resultats positifs emergent.

In [ ]:
-- Preferences unimodales (single-peaked)
-- L'espace des alternatives est ordonne (ex: gauche-droite sur [0, 1])

def SinglePeaked {A : Type} [LinearOrder A] (p : Preference A) (peak : A) : Prop :=
  -- A gauche du pic : plus on approche du pic, mieux c'est
  (∀ x y : A, x < y → y ≤ peak → p.R y x) ∧
  -- A droite du pic : plus on s'eloigne du pic, moins bon c'est
  (∀ x y : A, peak ≤ x → x < y → p.R x y)

#check @SinglePeaked

-- Un profil est unimodal si tous les electeurs ont des preferences unimodales
def SinglePeakedProfile {I A : Type} [LinearOrder A] (prefs : Profile I A) : Prop :=
  ∀ i : I, ∃ peak : A, SinglePeaked (prefs i) peak

#check @SinglePeakedProfile

### 5.2 Theoreme de l'electeur median

**Theoreme (Black 1948)** : Si tous les electeurs ont des preferences unimodales sur un espace unidimensionnel, alors sous vote majoritaire, l'alternative preferee de l'**electeur median** est le **vainqueur de Condorcet**.

In [ ]:
-- Theoreme de l'electeur median (enonce)

theorem median_voter_theorem_sketch
    {I A : Type} [Fintype I] [LinearOrder A]
    (prefs : Profile I A)
    (h_single_peaked : SinglePeakedProfile prefs)
    (h_odd : Odd (Fintype.card I)) :
    -- Il existe un gagnant de Condorcet
    ∃ winner : A, ∀ alt : A, alt ≠ winner →
      -- Plus de la moitie prefere winner a alt
      True := by
  sorry

#check @median_voter_theorem_sketch

---

## 6. Exercices

### Exercice 1 : Dictature et axiomes

Montrer qu'une dictature satisfait les axiomes de Pareto faible et IIA.

### Exercice 2 : Deux alternatives

Montrer que le theoreme d'Arrow ne s'applique pas avec seulement 2 alternatives.

### Exercice 3 : Preuve de Pareto

Prouver formellement que l'ensemble de tous les individus est decisif.

---

## 7. Solutions

### Solution Exercice 1

In [ ]:
-- Solution Exercice 1 : Une dictature satisfait Pareto et IIA

-- Construire une dictature
def dictatorship {I A : Type} (d : I) : SocialWelfareFunction I A := {
  f := fun prefs => prefs d
}

-- Une dictature satisfait Pareto faible
theorem dictatorship_pareto {I A : Type} (d : I) : 
    WeakPareto (dictatorship d (I := I) (A := A)) := by
  intro prefs x y h_all
  exact h_all d

-- Une dictature satisfait IIA
theorem dictatorship_iia {I A : Type} (d : I) :
    IIA (dictatorship d (I := I) (A := A)) := by
  intro prefs prefs2 x y h_xy h_yx
  simp only [dictatorship]
  exact h_xy d

#check @dictatorship_pareto
#check @dictatorship_iia

### Solution Exercice 2

Avec 2 alternatives, la regle majoritaire satisfait Pareto, IIA et Non-dictature. Le theoreme d'Arrow requiert |A| >= 3. Voir le notebook Python compagnon (20b) pour l'illustration.

---

## Resume

### Theoremes formalises

| Theoreme | Hypotheses | Conclusion |
|----------|------------|------------|
| **Arrow** | |A| >= 3, Pareto, IIA | Dictature |
| **Sen** | |A| >= 4, Pareto, Liberte | Contradiction |
| **Electeur Median** | Preferences unimodales, impair | Gagnant Condorcet existe |

### Definitions cles

| Concept | Definition Lean |
|---------|----------------|
| Preference | `Preference A` avec `R`, `complete`, `trans` |
| Profil | `Profile I A := I → Preference A` |
| SWF | `SocialWelfareFunction I A` avec `f : Profile I A → Preference A` |
| Pareto | `WeakPareto` : unanimite stricte → social strict |
| IIA | `IIA` : preference pairwise suffit |
| Non-dictature | `NonDictatorial` : pas de dictateur |
| SinglePeaked | `SinglePeaked` : preference unimodale avec pic |

### References

- [asouther4/lean-social-choice](https://github.com/asouther4/lean-social-choice) - Formalisation Lean 3 originale
- Arrow, K. (1951). "Social Choice and Individual Values"
- Sen, A. (1970). "The Impossibility of a Paretian Liberal"
- Black, D. (1948). "On the Rationale of Group Decision-making"

---

**Navigation** : [← GameTheory-19-Lean-CombinatorialGames](GameTheory-19-Lean-CombinatorialGames.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-21-Lean-CooperativeGames →](GameTheory-21-Lean-CooperativeGames.ipynb)